In [40]:
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes

print('Libraries installed!')


Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.7.1       |        py36_1001         140 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    certifi-2019.3.9           |           py36_0         149 KB  conda-forge
    conda-4.6.8                |           py36_0         876 KB  conda-forge
    openssl-1.1.1b             |       h14c3975_1         4.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.2 MB

The following packages will be UPDATED:

  beautifulsoup4      anaconda::beautifulsoup4-4.7.1-py36_1 --> conda-forge::beautifulsoup4-4.7.1-py36_1001
  ca-certificates                     2018.

In [341]:
from bs4 import BeautifulSoup
import requests
import lxml

import numpy as np
import pandas as pd

import requests
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim

import folium
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.preprocessing import StandardScaler, normalize, scale
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score

print('Libraries imported!')

Libraries imported!


In [342]:
#Check list of boroughs in Stockholm from Stockholm.se
#Grab population data
#http://statistik.stockholm.se/temp_eng/a-tabeller/tv.07437.html

# WE WANT TO COMPARE 2 BOROUGHS IN THE CITY CENTER

boroughs = pd.DataFrame()

#First Östermalm - grab longitude and latitude data

wikipedia_link='https://en.wikipedia.org/wiki/' + '%C3%96stermalm_(borough)'
raw_random_wikipedia_page =requests.get(wikipedia_link)
page = raw_random_wikipedia_page.text
soup = BeautifulSoup(page, 'lxml')
geoTag=soup.find(True,'geo')
(lat,lon) = geoTag.string.split(';')

boroughs = boroughs.append({'Borough': 'Ostermalm', 'Population': int(66521),'Latitude': float(lat), 'Longitude': float(lon)},ignore_index=True)

#Second Södermalm - grab longitude and latitude data

wikipedia_link='https://en.wikipedia.org/wiki/' + 'S%C3%B6dermalm_(borough)'
raw_random_wikipedia_page =requests.get(wikipedia_link)
page = raw_random_wikipedia_page.text
soup = BeautifulSoup(page, 'lxml')
geoTag=soup.find(True,'geo')
(lat,lon) = geoTag.string.split(';')

boroughs = boroughs.append({'Borough': 'Sodermalm', 'Population': 123468,'Latitude': float(lat), 'Longitude': float(lon)},ignore_index=True)

boroughs.Latitude = df.Latitude.astype(float)
boroughs.Longitude = df.Longitude.astype(float)
boroughs.Population = df.Population.astype(int)

print(boroughs)

     Borough  Latitude  Longitude  Population
0  Ostermalm  59.33814   18.08672       66521
1  Sodermalm  59.31000   18.06000      123468


In [343]:
CLIENT_ID = 'XK02GR2LEMSL0GFCHQQ2QWUVW3XWINN1UL2WUIG4LUUM0QNY' # your Foursquare ID
CLIENT_SECRET = 'XYPJEZIDOC12IOUKW3KYGTWMVEFUZV2GPXX3IZPFMVJJ0L1Z' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: XK02GR2LEMSL0GFCHQQ2QWUVW3XWINN1UL2WUIG4LUUM0QNY
CLIENT_SECRET:XYPJEZIDOC12IOUKW3KYGTWMVEFUZV2GPXX3IZPFMVJJ0L1Z


In [373]:
address = 'Stockholm, Sweden'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Stockholm, Sweden are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Stockholm, Sweden are 59.3251172, 18.0710935.


In [374]:
# create map of Stockholm using latitude and longitude values
map_stockholm = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map denoting locations of interest (Ostermalm and Sodermalm)
for lat, lng, borough in zip(boroughs['Latitude'], boroughs['Longitude'], boroughs['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_stockholm)  
    
map_stockholm

In [375]:
#First pull data for Ostermalm

lat = boroughs['Latitude'].loc[boroughs['Borough'].isin(['Ostermalm'])].sum()
lon = boroughs['Longitude'].loc[boroughs['Borough'].isin(['Ostermalm'])].sum()

LIMIT = 1000 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lon, 
    radius, 
    LIMIT)
url # display URL
results = requests.get(url).json()

In [376]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [377]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

400

In [349]:
# create map of Stockholm using latitude and longitude values
map_stockholm = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map (all venues in Ostermalm)
for lat, lng, borough in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_stockholm)  
    
map_stockholm

In [350]:
competition_venues = nearby_venues[nearby_venues['categories'].str.contains('Coffee Shop|Bistro|Café')]

attraction_venues = nearby_venues[nearby_venues['categories'].str.contains('Bookstore|Movie Theater|Gym / Fitness Center|'
                                                       'Plaza|Dance Studio|Food Court|Theater|Boutique|'
                                                       'Antique Shop|Opera House|History Museum|Shoe Store|Yoga Studio|'
                                                      'Hotel|Park|Stadium|Concert Hall|Furniture / Home Store|Theme Park|'
                                                       'Women\'s Store|Museum|Boutique|Department Store|Art Museum|Office')]


# create map of Stockholm using latitude and longitude values
map_stockholm = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map (attractions as blue)
for lat, lng, name in zip(attraction_venues['lat'], attraction_venues['lng'], attraction_venues['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_stockholm)  
    
# add markers to map (competition as red)
for lat, lng, name in zip(competition_venues['lat'], competition_venues['lng'], competition_venues['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_stockholm)  
    
map_stockholm

In [351]:
print('There are {} uniques categories.'.format(len(nearby_venues['categories'].unique())))

There are 52 uniques categories.


In [352]:
dfx=nearby_venues['categories'].unique()
print(dfx)


['Pizza Place' 'Italian Restaurant' 'Scandinavian Restaurant'
 'French Restaurant' 'Hot Dog Joint' 'Seafood Restaurant' 'Theater'
 'History Museum' 'Shoe Store' 'Gourmet Shop' 'Persian Restaurant'
 'Spanish Restaurant' 'Yoga Studio' 'Café' 'Bakery'
 'Modern European Restaurant' 'Beer Bar' 'Design Studio'
 'Asian Restaurant' 'Hotel' 'Park' 'Coffee Shop' 'Lounge' 'Stadium'
 'Concert Hall' 'Pool' 'Furniture / Home Store' 'Gym / Fitness Center'
 'Cocktail Bar' 'Playground' 'Theme Park' "Women's Store" 'Museum'
 'Dog Run' 'Smoke Shop' 'Movie Theater' 'Clothing Store' 'Restaurant'
 'Szechuan Restaurant' 'Fried Chicken Joint' 'Tapas Restaurant' 'Boutique'
 'Steakhouse' 'Burger Joint' 'Medical Center' 'Salad Place'
 'Department Store' 'Food Court' 'Organic Grocery' 'Sporting Goods Shop'
 'Office' 'Art Museum']


In [353]:
nearby_venues['categories'].value_counts()


Scandinavian Restaurant       18
Hotel                          8
Seafood Restaurant             4
Café                           4
History Museum                 3
Italian Restaurant             3
Coffee Shop                    3
Park                           3
Asian Restaurant               2
Burger Joint                   2
Bakery                         2
Movie Theater                  2
French Restaurant              2
Theater                        2
Gym / Fitness Center           2
Lounge                         2
Clothing Store                 2
Museum                         2
Pool                           1
Boutique                       1
Beer Bar                       1
Office                         1
Design Studio                  1
Department Store               1
Playground                     1
Women's Store                  1
Food Court                     1
Modern European Restaurant     1
Theme Park                     1
Hot Dog Joint                  1
Szechuan R

In [371]:

competition = nearby_venues['categories'].str.contains('Coffee Shop|Bistro|Café').value_counts()[True]
competition

14

In [355]:
attractions = nearby_venues['categories'].str.contains('Bookstore|Movie Theater|Gym / Fitness Center|'
                                                       'Plaza|Dance Studio|Food Court|Theater|Boutique|'
                                                       'Antique Shop|Opera House|History Museum|Shoe Store|Yoga Studio|'
                                                      'Hotel|Park|Stadium|Concert Hall|Furniture / Home Store|Theme Park|'
                                                       'Women\'s Store|Museum|Boutique|Department Store|Art Museum|Office').value_counts()[True]
attractions

34

In [356]:
boroughs.loc[0,'Competition'] = competition
boroughs.loc[0,'Attractions'] = attractions
print(boroughs)

     Borough  Latitude  Longitude  Population  Competition  Attractions
0  Ostermalm  59.33814   18.08672       66521          7.0         34.0
1  Sodermalm  59.31000   18.06000      123468          NaN          NaN


In [357]:
#First pull data for Sodermalm

lat = boroughs['Latitude'].loc[boroughs['Borough'].isin(['Sodermalm'])].sum()
lon = boroughs['Longitude'].loc[boroughs['Borough'].isin(['Sodermalm'])].sum()

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lon, 
    radius, 
    LIMIT)
url # display URL
results = requests.get(url).json()

In [358]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Årstaviken,Lake,59.305379,18.063498
1,Skånegläntan,Playground,59.311211,18.069953
2,Johan & Nyström,Coffee Shop,59.316273,18.064016
3,Morfar Ginko/Pappa Ray Ray,Bar,59.315774,18.064144
4,Shanti Softcorner,Indian Restaurant,59.313582,18.067843


In [359]:
# create map of Stockholm using latitude and longitude values
map_stockholm = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map (all venues in Sodermalm)
for lat, lng, borough in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_stockholm)  
    
map_stockholm

In [360]:
competition_venues = nearby_venues[nearby_venues['categories'].str.contains('Coffee Shop|Bistro|Café')]

attraction_venues = nearby_venues[nearby_venues['categories'].str.contains('Bookstore|Movie Theater|Gym / Fitness Center|'
                                                       'Plaza|Dance Studio|Food Court|Theater|Boutique|'
                                                       'Antique Shop|Opera House|History Museum|Shoe Store|Yoga Studio|'
                                                      'Hotel|Park|Stadium|Concert Hall|Furniture / Home Store|Theme Park|'
                                                       'Women\'s Store|Museum|Boutique|Department Store|Art Museum|Office')]


# create map of Stockholm using latitude and longitude values
map_stockholm = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map (attractions as blue)
for lat, lng, name in zip(attraction_venues['lat'], attraction_venues['lng'], attraction_venues['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_stockholm)  
    
# add markers to map (competition as red)
for lat, lng, name in zip(competition_venues['lat'], competition_venues['lng'], competition_venues['name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_stockholm)  
    
map_stockholm

In [361]:
print('There are {} uniques categories.'.format(len(nearby_venues['categories'].unique())))

There are 57 uniques categories.


In [362]:
dfx=nearby_venues['categories'].unique()
print(dfx)

['Lake' 'Playground' 'Coffee Shop' 'Bar' 'Indian Restaurant' 'Bakery'
 'Bookstore' 'Café' 'Cocktail Bar' 'Tapas Restaurant' 'Asian Restaurant'
 'Vegetarian / Vegan Restaurant' 'Falafel Restaurant'
 'Arts & Crafts Store' 'Movie Theater' 'Plaza' 'Dance Studio'
 'Scandinavian Restaurant' 'Hotel' 'Bistro' 'Greek Restaurant'
 'Organic Grocery' 'Czech Restaurant' 'Ice Cream Shop' 'Restaurant'
 'Boutique' 'Shoe Store' 'Theater' 'Chinese Restaurant' 'Creperie'
 'Record Shop' 'Deli / Bodega' 'Candy Store' 'Food Court' 'Pool'
 'Clothing Store' 'Beer Bar' 'Thai Restaurant' 'Italian Restaurant'
 'Beer Garden' 'Scenic Lookout' 'Park' 'Pub' 'Nightclub' 'Burger Joint'
 'Mountain' 'Design Studio' 'Middle Eastern Restaurant'
 'Hawaiian Restaurant' 'Taco Place' 'Dive Bar' 'Electronics Store'
 'Sushi Restaurant' 'Kebab Restaurant' 'Noodle House' 'Gym' 'Outdoor Gym']


In [363]:
nearby_venues['categories'].value_counts()

Coffee Shop                      7
Bar                              6
Café                             6
Vegetarian / Vegan Restaurant    5
Bookstore                        4
Indian Restaurant                3
Thai Restaurant                  3
Bakery                           3
Falafel Restaurant               3
Scandinavian Restaurant          3
Cocktail Bar                     2
Pub                              2
Chinese Restaurant               2
Beer Garden                      2
Deli / Bodega                    2
Park                             2
Theater                          2
Playground                       2
Hotel                            2
Clothing Store                   2
Boutique                         1
Gym                              1
Design Studio                    1
Greek Restaurant                 1
Middle Eastern Restaurant        1
Sushi Restaurant                 1
Pool                             1
Record Shop                      1
Dive Bar            

In [364]:
competition = nearby_venues['categories'].str.contains('Coffee Shop|Bistro|Café').value_counts()[True]
competition


14

In [365]:
attractions = nearby_venues['categories'].str.contains('Bookstore|Movie Theater|Gym / Fitness Center|'
                                                       'Plaza|Dance Studio|Food Court|Theater|Boutique|'
                                                       'Antique Shop|Opera House|History Museum|Shoe Store|Yoga Studio|'
                                                      'Hotel|Park|Stadium|Concert Hall|Furniture / Home Store|Theme Park|'
                                                       'Women\'s Store|Museum|Boutique|Department Store|Art Museum|Office').value_counts()[True]
attractions

16

In [366]:
boroughs.loc[1,'Competition'] = competition
boroughs.loc[1,'Attractions'] = attractions
print(boroughs)

     Borough  Latitude  Longitude  Population  Competition  Attractions
0  Ostermalm  59.33814   18.08672       66521          7.0         34.0
1  Sodermalm  59.31000   18.06000      123468         14.0         16.0
